# Advanced Training Metrics and Formulations

This notebook provides a detailed walkthrough of the key metrics and mathematical formulations used in our ensemble training pipeline. In this notebook, we cover:

- **Token‑Level and Sequence‑Level Entropy:** Measuring uncertainty in model predictions.
- **Logistic Regression Ensemble:** Combining features such as model confidences, entropy, and KNN similarity to generate a final quality score.
- **FAISS‑based KNN Retrieval:** Fast retrieval of similar examples using normalized embeddings and cosine similarity.
- **Cross‑Model Alignment Loss:** Optimizing the shared latent space between models with adversarial perturbation.
- **Knowledge Distillation:** Training a compact student model to mimic the ensemble output.

The notebook also includes visualizations of training loss and entropy trends over epochs.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

# For LaTeX rendering in markdown cells
from IPython.display import display, Math

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

## Entropy Calculations

For a probability distribution \(\mathbf{p}\), the token-level entropy is defined as:

$$ H(\mathbf{p}) = -\sum_{i} p_i \log p_i $$

This measure is then aggregated over the sequence. For example, the mean entropy over a sequence of \(n\) tokens is:

$$ H_{seq} = \frac{1}{n} \sum_{i=1}^{n} H(p_i) $$

In [ ]:
def calculate_token_entropy(logits, temperature=1.0):
    """Calculate token-level entropy from logits with temperature scaling."""
    scaled_logits = logits / temperature
    probs = F.softmax(scaled_logits, dim=-1)
    log_probs = torch.log(probs + 1e-10)
    token_entropy = -torch.sum(probs * log_probs, dim=-1)
    return token_entropy

def aggregate_sequence_entropy(token_entropies, reduction='mean'):
    """Aggregate token-level entropy to a single sequence-level value."""
    if reduction == 'mean':
        return token_entropies.mean()
    elif reduction == 'max':
        return token_entropies.max()
    elif reduction == 'weighted_mean':
        weights = torch.arange(1, token_entropies.size(0) + 1).float() / token_entropies.size(0)
        return (token_entropies * weights).sum() / weights.sum()
    else:
        raise ValueError(f"Unknown reduction method: {reduction}")

# Simulate logits for demonstration
dummy_logits = torch.randn(5, 100)  # Batch size 5, sequence length 100
token_entropy = calculate_token_entropy(dummy_logits)
sequence_entropy = aggregate_sequence_entropy(token_entropy)
print('Mean Sequence Entropy:', sequence_entropy.item())

## Logistic Regression Ensemble

The ensemble logistic regression layer fuses multiple features to output a final quality score. Given the feature vector \( \mathbf{x} \in \mathbb{R}^7 \), the logistic regression prediction is defined as:

$$ \hat{y} = \sigma(\mathbf{w}^T \mathbf{x} + b) $$

where \( \sigma(z) = \frac{1}{1+e^{-z}} \) is the sigmoid function. The features include:

- **f1:** Primary confidence (\(1 - H_{seq}\))
- **f2:** Secondary confidence (based on ensemble disagreement)
- **f3:** Evaluator confidence (\(1 - \) evaluator entropy)
- **f4:** Raw primary sequence entropy
- **f5:** Ensemble disagreement
- **f6:** KNN similarity score
- **f7:** Example quality metadata


In [ ]:
import torch.nn as nn

class EnsembleLogisticRegression(nn.Module):
    def __init__(self, feature_dim=7):
        super(EnsembleLogisticRegression, self).__init__()
        self.logistic = nn.Linear(feature_dim, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, features):
        logits = self.logistic(features)
        return self.sigmoid(logits)

# Simulate a feature vector
features = torch.tensor([[0.8, 0.7, 0.9, 0.2, 0.3, 0.85, 1.0]])
ensemble_lr = EnsembleLogisticRegression()
score = ensemble_lr(features)
print('Ensemble Prediction Score:', score.item())

## FAISS-based KNN Retrieval

To quickly retrieve similar code examples, we use a FAISS-based index. The cosine similarity between normalized embeddings \(\mathbf{u}\) and \(\mathbf{v}\) is computed as:

$$ \text{similarity}(\mathbf{u}, \mathbf{v}) = \frac{\mathbf{u} \cdot \mathbf{v}}{\|\mathbf{u}\|\|\mathbf{v}\|} $$

High similarity indicates that the retrieved example is very similar to the query.

In [ ]:
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class KNNIndex:
    def __init__(self, embedding_dim):
        self.embedding_dim = embedding_dim
        self.index = faiss.IndexFlatIP(embedding_dim)  # Inner product for cosine similarity
        self.examples = []
        self.metadata = []
    def add_example(self, text, embedding, metadata=None):
        if embedding.shape[0] != self.embedding_dim:
            raise ValueError(f"Expected embedding dimension {self.embedding_dim}, got {embedding.shape[0]}")
        self.index.add(np.array([embedding]).astype('float32'))
        self.examples.append(text)
        self.metadata.append(metadata or {})
    def search(self, query_embedding, k=5):
        query_embedding = np.array([query_embedding]).astype('float32')
        distances, indices = self.index.search(query_embedding, k)
        results = []
        for i, idx in enumerate(indices[0]):
            if idx != -1:
                results.append({
                    'text': self.examples[idx],
                    'distance': distances[0][i],
                    'metadata': self.metadata[idx]
                })
        return results

# Simulate adding and retrieving examples
knn = KNNIndex(embedding_dim=384)
example_embedding = np.random.rand(384).astype('float32')
knn.add_example("def example(): pass", example_embedding, {"quality": 0.95})
results = knn.search(example_embedding, k=1)
print('KNN Retrieval Results:', results)

## Cross‑Model Alignment Loss

To align the latent spaces of different models, we project their final hidden states into a common space and compute cosine similarity. The alignment loss is computed as the average of the disagreements:

$$ \mathcal{L}_{align} = \frac{1}{3} \Bigl( (1 - \cos(\mathbf{z}_1, \mathbf{z}_2)) + (1 - \cos(\mathbf{z}_1, \mathbf{z}_3)) + (1 - \cos(\mathbf{z}_2, \mathbf{z}_3)) \Bigr) $$

where \( \mathbf{z}_1, \mathbf{z}_2, \mathbf{z}_3 \) are the projected representations of the primary, secondary, and evaluator models respectively.

In [ ]:
# This cell would normally run the cross-model alignment in the training loop.
# Here we simulate random projected vectors for demonstration:
z1 = F.normalize(torch.randn(1, 512), p=2, dim=1)
z2 = F.normalize(torch.randn(1, 512), p=2, dim=1)
z3 = F.normalize(torch.randn(1, 512), p=2, dim=1)

cos12 = F.cosine_similarity(z1, z2).item()
cos13 = F.cosine_similarity(z1, z3).item()
cos23 = F.cosine_similarity(z2, z3).item()

alignment_loss = ( (1 - cos12) + (1 - cos13) + (1 - cos23) ) / 3.0
print(f"Alignment Loss: {alignment_loss:.4f}")

## Advanced Knowledge Distillation

In our system, a student MLP is trained to mimic the output of the ensemble projection (e.g., the output from the primary model’s projection layer). The training objective is to minimize the MSE loss between the student’s prediction and the target projection:

$$ \mathcal{L}_{distill} = \| f_{student}(\mathbf{h}) - f_{target}(\mathbf{h}) \|^2 $$

where \( \mathbf{h} \) represents the hidden states from the primary model.

## Final Visualization

Below we visualize simulated training loss and entropy metrics over epochs to monitor training progress.

In [ ]:
# Simulate training metrics
epochs = np.arange(1, 11)
loss = np.exp(-epochs / 3) + np.random.normal(0, 0.05, size=10)
entropy = np.linspace(4.5, 3.0, 10) + np.random.normal(0, 0.1, size=10)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ax[0].plot(epochs, loss, marker='o')
ax[0].set_title('Simulated Training Loss')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')

ax[1].plot(epochs, entropy, marker='o', color='orange')
ax[1].set_title('Simulated Sequence Entropy')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Entropy')

plt.tight_layout()
plt.show()